In [2]:
%pip --version

pip 22.0.2 from /home/rian/Programming/Python_progs/Cursos/F9_CV/C6-AnaliseFacial/fcv6-expressoes/.venv/lib/python3.10/site-packages/pip (python 3.10)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install opencv-python==4.6.0.66 mediapipe==0.8.11 numpy==1.22.3 # DO CURSO     
# %pip install -U opencv-python mediapipe numpy (O QUE USEI)

# VERSÕES USADAS
# opencv-contrib-python==4.11.0.86
# opencv-python==4.11.0.86
# numpy==1.26.4
# mediapipe==0.10.21

In [62]:
import cv2
import mediapipe as mp
import numpy as np
import time

## <font color='limegreen'> Parte 1</font>

In [23]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

In [45]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


# (nível minímo de confiança para detectar uma face, mínimo de confiança para detectar os pontos da face)
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5, refine_landmarks=True) as facemesh:
    while cap.isOpened():
        tecla = cv2.waitKey(1)
        if tecla == 27: # Esc
            break


        sucesso, frame = cap.read()
        if not sucesso:
            print('Ignorando o frame vazio da camera')
            continue

        # OpenCV por padrão entrega dados em BGR
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        saida_facemesh = facemesh.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        try:
            # Itera sobre cada ponto da face coletado
            for face_landmarks in saida_facemesh.multi_face_landmarks:
                # Frame (fundo), pontos da face, drawing options
                mp_drawing.draw_landmarks(
                    frame, 
                    face_landmarks, 
                    mp_face_mesh.FACEMESH_CONTOURS,
                    # Blue, Green, Red (tonalidade tendendo pro Azul)
                    landmark_drawing_spec=mp_drawing.DrawingSpec(
                        color=(255,102,0),
                        thickness=1, 
                        circle_radius=1
                    ), 
                    # (tonalidade tendendo para o verde), espessura dos traços
                    connection_drawing_spec=mp_drawing.DrawingSpec(
                        color=(102,204,0), 
                        thickness=1,
                        circle_radius=1
                    )
                )
                
        except Exception as e:
            # Caso ocorra algum erro, como por exemplo, a obstrução do rosto
            print(e)
            pass

        cv2.imshow('Camera', frame) # Não colocar acento no nome da imagem!

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1743270536.686741   30440 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743270536.687724  125862 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
W0000 00:00:1743270536.691159  125852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743270536.705926  125855 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable


## <font color='limegreen'> Parte 2 - Mapeando Olhos</font>

In [ ]:
saida_facemesh.multi_face_landmarks

# [landmark {
#    x: 0.55995214
#    y: 0.746125638
#    z: -0.0287616495
#  }
#  landmark {
#    x: 0.556740046
#    y: 0.683440685
#    z: -0.0586610846
#  }
# ...
# Ao todo, 468 pontos coletados

In [40]:
type(saida_facemesh.multi_face_landmarks) # list (de len=1)
type(saida_facemesh.multi_face_landmarks[0])
# mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList
saida_facemesh.multi_face_landmarks[0].landmark
# [x: 0.632665
# y: 0.981715679
# z: -0.0286340564
# , x: 0.633395314
# y: 0.919632852
# z: -0.0591587462
# , x: 0.630356848
# ...
type(saida_facemesh.multi_face_landmarks[0].landmark)
# google._upb._message.RepeatedCompositeContainer


google._upb._message.RepeatedCompositeContainer

In [ ]:
for face_landmarks in saida_facemesh.multi_face_landmarks:
    face = face_landmarks
    for id_coord, coord_xyz in enumerate(face.landmark):
        print(str(id_coord) + '\n', coord_xyz)

# 0
#  x: 0.632665
# y: 0.981715679
# z: -0.0286340564

# 1
#  x: 0.633395314
# y: 0.919632852
# z: -0.0591587462

# ...
# 467
#  x: 0.702854097
# y: 0.758263469
# z: 0.0227333456

<hr color='orange'>

In [ ]:
p_olho_esq = [385, 380, 387, 373, 362, 263] # já estão em sequência
p_olho_dir = [160, 144, 158, 153, 33, 133] # já estão em sequência
p_olhos = p_olho_esq + p_olho_dir

In [59]:
def calculo_ear(face, p_olho_dir, p_olho_esq):
    # Para evitar problemas como divisão por 0
    try: 
        face = np.array([[coord.x, coord.y] for coord in face]) # cria uma lista com apenas 2 das dimensões coletadas pelo mediapipe
        # Coleta apenas os parâmetros de face do olho esquerdo e direito
        face_esq = face[p_olho_esq,:]
        face_dir = face[p_olho_dir,:]

        # np.linalg.norm: Calcula a distância euclidiana | os pontos já estão na ordem, mas deveria ser: (||p2 - p6|| + ||p3 - p5||)/(2 * ||p1 - p4||)
        ear_esq = (np.linalg.norm(face_esq[0] - face_esq[1]) + np.linalg.norm(face_esq[2] - face_esq[3])) / (2 * np.linalg.norm(face_esq[0] - face_esq[1]))
        ear_dir = (np.linalg.norm(face_dir[0] - face_dir[1]) + np.linalg.norm(face_dir[2] - face_dir[3])) / (2 * np.linalg.norm(face_dir[0] - face_dir[1]))
    except Exception as e:
        print(e)
        ear_esq = 0
        ear_dir = 0
    
    media_ear = (ear_esq + ear_dir)/2
    return media_ear

In [ ]:
ear_limiar = 1.005 # Entre 1.000 e 1.010 no meu caso

In [99]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


dormindo = 0 # Não está dormindo


# (nível minímo de confiança para detectar uma face, mínimo de confiança para detectar os pontos da face)
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5, refine_landmarks=True) as facemesh:
    while cap.isOpened():
        tecla = cv2.waitKey(1)
        if tecla == 27: # Esc
            break


        sucesso, frame = cap.read()
        if not sucesso:
            print('Ignorando o frame vazio da camera')
            continue

        # 3ª var: FPS
        comprimento, largura, _ = frame.shape # para o _normalized_to_pixel_coordenates()

        # OpenCV por padrão entrega dados em BGR
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        saida_facemesh = facemesh.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        try:
            # Itera sobre cada ponto da face coletado
            for face_landmarks in saida_facemesh.multi_face_landmarks:
                # Frame (fundo), pontos da face, drawing options
                mp_drawing.draw_landmarks(
                    frame, 
                    face_landmarks, 
                    mp_face_mesh.FACEMESH_CONTOURS,
                    # Blue, Green, Red (tonalidade tendendo pro Azul)
                    landmark_drawing_spec=mp_drawing.DrawingSpec(
                        color=(255,102,0),
                        thickness=1, 
                        circle_radius=1
                    ), 
                    # (tonalidade tendendo para o verde), espessura dos traços
                    connection_drawing_spec=mp_drawing.DrawingSpec(
                        color=(102,204,0), 
                        thickness=1,
                        circle_radius=1
                    )
                )

                face = face_landmarks.landmark
                for id_coord, coord_xyz in enumerate(face):
                    # Verifica se as coordenadas são de um ponto com id pertencente à lista referente aos olhos
                    if id_coord in p_olhos:
                        # Faz a conversão de coordenadas normalizadas para pixels
                        coord_cv = mp_drawing._normalized_to_pixel_coordinates(
                            coord_xyz.x, coord_xyz.y, largura, comprimento)
                        cv2.circle(frame, coord_cv, radius=2, color=(255,0,0), thickness=-1)
                        
                ear = calculo_ear(face, p_olho_dir, p_olho_esq)
                # Para ter um fundo e criar contraste para o texto
                # Ponto inicial | Ponto final do retângulo | cor (cinza) | thickness=-1 (retângulo preenchido)
                cv2.rectangle(
                    frame, pt1=(0,1), pt2=(290, 140), color=(58,58,55), thickness=-1)
                cv2.putText(
                    frame, 
                    text=f"EAR: {round(ear, 2)}", org=(1, 24),
                    fontFace=cv2.FONT_HERSHEY_DUPLEX,
                    fontScale=0.9, color=(255, 255, 255), thickness=2)
                
                if ear < ear_limiar:
                    # dormindo já começa como 0, logo não haveria o problema da primeira inicialização de t_inicial quando dormindo = 1
                    t_inicial = time.time() if dormindo == 0 else t_inicial
                    dormindo = 1
                if dormindo == 1 and ear >= ear_limiar:
                    # enquanto estiver com o olho fechado e o "ear" não for maior que o limiar (o olho ainda não abriu), não entra nesse caso para definir que o olho de fato abriu
                    dormindo = 0 # Não se está mais em descanso

                t_final = time.time()
                # Só conta o tempo se o olho estiver fechado (dormindo verdadeiro)
                tempo = (t_final - t_inicial) if dormindo == 1 else 0.0
                
                cv2.putText(
                    frame, 
                    text=f"Tempo: {round(tempo), 3}", org=(1, 80),
                    fontFace=cv2.FONT_HERSHEY_DUPLEX,
                    fontScale=0.9, color=(255, 255, 255), thickness=2)
                
                if tempo>=1.5:
                    cv2.rectangle(frame, pt1=(30, 400), pt2=(610, 452), color=(109, 233, 219), thickness=-1) # Preenche todo o retângulo
                    cv2.putText(
                        frame, 
                        text=f"Muito tempo com olhos fechados!", 
                        org=(80, 435), # cor do texto (mesmo cinza do retângulo)
                        fontFace=cv2.FONT_HERSHEY_DUPLEX,
                        fontScale=0.85, 
                        color=(58,58,55), 
                        thickness=1)
                
        except Exception as e:
            # Caso ocorra algum erro, como por exemplo, a obstrução do rosto
            print(e)
            pass

        cv2.imshow('Camera', frame) # Não colocar acento no nome da imagem!

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1743276634.637807   30440 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743276634.638485  166352 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
W0000 00:00:1743276634.641055  166346 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743276634.650427  166348 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
